In [1]:
import datetime
from dateutil import parser

In [2]:
parser.parse

<function dateutil.parser._parser.parse(timestr, parserinfo=None, **kwargs)>

In [5]:
p = parser.parse(timestr="2024-10-06T09:37:36.590Z", ignoretz=True)

In [4]:
n = datetime.datetime.now()

In [7]:
(n-p) > datetime.timedelta(days=90)

False

In [40]:
n.isoformat(timespec='milliseconds')

'2024-11-06T23:32:10.408'

In [43]:
datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S.%f")[:-3]

'2024-11-06 23:50:57.801'

In [16]:
d = n - datetime.timedelta(hours=8)

In [19]:
d 

datetime.datetime(2024, 11, 6, 15, 32, 10, 408230)

In [30]:
tmp = (n - d).seconds

In [31]:
tmp

28800

In [29]:
tmp.seconds/60/60

True

In [8]:
import yaml

In [9]:
with open('config.yaml') as f:
    config = yaml.safe_load(f)

In [10]:
config

{'index': 'docs',
 'scheduler': False,
 'least_n_days': 90,
 'platforms': {'ptt': {'boards': ['Gossiping',
    'Stock',
    'HatePolitics',
    'movie',
    'PC_Shopping']},
  'dcard': {'boards': ['youtuber',
    'talk',
    'trending',
    '3c',
    'creditcard',
    'makeup']}}}

In [14]:
config["platforms"]["ptt"]["boards"]

['Gossiping,', 'Stock,', 'HatePolitics,', 'movie,', 'PC_Shopping,']

In [3]:
type(config["least_n_days"])

int

In [6]:
config["platforms"]["dcard"]

{'boards': ['youtuber,', 'talk,', 'trending,', '3c,', 'creditcard,', 'makeup']}

In [ ]:
!pip install nlp2

In [6]:
from elasticsearch import Elasticsearch, helpers

client = Elasticsearch(
    "http://127.0.0.1:9200", verify_certs=False, basic_auth=("elastic", "123456")
)

In [7]:
client.inference.get(inference_id="tencentbac_conan_embedding_v1")

ObjectApiResponse({'endpoints': [{'inference_id': 'tencentbac_conan_embedding_v1', 'task_type': 'text_embedding', 'service': 'openai', 'service_settings': {'model_id': 'tencentbac_conan_embedding_v1', 'url': 'http://localai:8080/embeddings', 'similarity': 'dot_product', 'dimensions': 1792, 'rate_limit': {'requests_per_minute': 3000}}, 'task_settings': {}}]})

In [4]:
def inference_check(client: Elasticsearch):
    try:
        client.inference.get(inference_id="tencentbac_conan_embedding_v1")
    except Exception as e:
        print("Inference creating...")
        client.inference.put(
            task_type="text_embedding",
            inference_id="tencentbac_conan_embedding_v1",
            body={
                "service": "openai",
                "service_settings": {
                    "model_id": "tencentbac_conan_embedding_v1",
                    "url": "http://localai:8080/embeddings",
                    "api_key": "ignored",
                },
            },
        )

In [8]:
inference_check(client)

In [2]:
client.info()

ObjectApiResponse({'name': 'fc981c3d8139', 'cluster_name': 'docker-cluster', 'cluster_uuid': 'CW97Ie5ZSPKXZafOAxIPeQ', 'version': {'number': '8.15.3', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': 'f97532e680b555c3a05e73a74c28afb666923018', 'build_date': '2024-10-09T22:08:00.328917561Z', 'build_snapshot': False, 'lucene_version': '9.11.1', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'})

In [ ]:
client.inference.put()

In [ ]:
client.ingest.get_pipeline(id="tencentbac_conan_embedding_pipe")

In [ ]:
client.ingest.put_pipeline(
    id="tencentbac_conan_embedding_pipe",
    body={
        "processors": [
            {
                "inference": {
                    "model_id": "tencentbac_conan_embedding_v1",
                    "input_output": [
                        {"input_field": "title", "output_field": "title_vector"},
                        {"input_field": "context", "output_field": "context_vector"},
                    ],
                }
            }
        ]
    }
)